In [1]:
# !pip install 'zarr<3'
# !pip install timm
# !pip install openslide-python tiffslide
# !pip install pyarrow
# !pip install shapely
# !pip install scikit-image
# !pip install opencv-python

# # Install CuPy (will auto-detect CUDA version)
# #Install CuPy pre-built binary for CUDA 12.x
# !pip install cupy-cuda12x

In [2]:
import torch
print(f"PyTorch CUDA version: {torch.version.cuda}")

# Then install matching CuPy:
# CUDA 11.x → !pip install cupy-cuda11x
# CUDA 12.x → !pip install cupy-cuda12x

PyTorch CUDA version: 12.1


In [3]:

# Verify CuPy installation
import cupy as cp
print(f'✅ CuPy installed: {cp.cuda.runtime.getDeviceCount()} GPU(s) available')

✅ CuPy installed: 1 GPU(s) available


In [4]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## Flex 

In [5]:
import torch
from vitaminp import VitaminPFlex
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold21_best.pth", map_location=device))
model.eval()

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


VitaminPFlex(
  (backbone): DINOv2Backbone(
    (dinov2): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): M

In [14]:
# Create channel config for Xenium data
config = ChannelConfig(
    nuclear_channel=0,           # Channel 0: DAPI
    membrane_channel=[1],        # Channel 1: CD45/E-cadherin
    membrane_combination='max',  # Not really needed for single membrane channel
    channel_names={0: 'DAPI', 1: 'CD45_Ecadherin', 2: 'Panck'}
)

print("\nChannel configuration:")
print(f"  Nuclear channel: 0 (DAPI)")
print(f"  Membrane channel: 1 (CD45/E-cadherin)")

# Create predictor
predictor = WSIPredictor(
    model=model,
    device=device,
    patch_size=512,
    overlap=64,
    target_mpp=0.4004,          # Xenium pixel size from your original code
    magnification=40,
    mif_channel_config=config
)

print("\nPredictor settings:")
print(f"  Patch size: 512")
print(f"  Overlap: 64")
print(f"  Target MPP: 0.2125 μm/pixel")
print(f"  Magnification: 40x")

# Run inference on combined 2-channel crop'

input_path = 'test_images/tile_dapi_merged.ome.tif'
## /rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/DSP Scans/MS010S1.ome.tiff

print(f"\n{'='*60}")
print("RUNNING INFERENCE...")
print(f"{'='*60}")
print(f"Input: {input_path}")

results = predictor.predict(
    wsi_path=input_path,
    output_dir='results_new',
    branch='mif_cell',              # Use cell branch for whole cells
    filter_tissue=False,
    tissue_threshold=0.01,# Process entire crop
    clean_overlaps=True,            # Clean boundary artifacts
    save_geojson=True,              # Save GeoJSON for visualization
    detection_threshold=0.3,        # Adjust for sensitivity
    min_area_um=10.0,               # Filter small artifacts (10 μm²)
    # mpp_override=0.263,            # Use Xenium pixel size
)

print(f"\n{'='*60}")
print("RESULTS")
print(f"{'='*60}")
print(f"✅ Found {results['num_detections']} cells in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")


2026-01-28 22:33:54 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-28 22:33:54 - WSIPredictor - INFO -   Device: cuda
2026-01-28 22:33:54 - WSIPredictor - INFO -   Model type: VitaminPFlex (single-modality)
2026-01-28 22:33:54 - WSIPredictor - INFO -   Patch size: 512
2026-01-28 22:33:54 - WSIPredictor - INFO -   Overlap: 64
2026-01-28 22:33:54 - WSIPredictor - INFO -   Magnification: 40
2026-01-28 22:33:54 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: max(CD45_Ecadherin)
2026-01-28 22:33:54 - WSIPredictor - INFO -    Using default MPP: 0.4004 μm/px
2026-01-28 22:33:54 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-28 22:33:54 - WSIPredictor - INFO -    WSI MPP: 0.4004 μm/px
2026-01-28 22:33:54 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-28 22:33:54 - WSIPredictor - INFO -    Scale factor: 1.52x
2026-01-28 22:33:54 - WSIPredictor - INFO -    Min area filter: 10.0 μm² = 62 pixels²
2026-01-28 22:33:54 - WSIPredictor - INFO - 📁


Channel configuration:
  Nuclear channel: 0 (DAPI)
  Membrane channel: 1 (CD45/E-cadherin)

Predictor settings:
  Patch size: 512
  Overlap: 64
  Target MPP: 0.2125 μm/pixel
  Magnification: 40x

RUNNING INFERENCE...
Input: test_images/tile_dapi_merged.ome.tif
Output: shape=(2, 3000, 3000), dtype=float32, range=[0.000, 0.689]
   Virtual upscaled size: 4567x4567 (from 3000x3000)
   Scanning 11x11 tile grid...


2026-01-28 22:33:54 - WSIPredictor - INFO -    ✓ Created 121 tiles (11x11 grid)
2026-01-28 22:33:54 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on mif_cell...
Processing tiles: 100%|██████████| 121/121 [00:31<00:00,  3.88it/s]
2026-01-28 22:34:25 - WSIPredictor - INFO -    DEBUG: Actually processed 121 tiles out of 121 available
2026-01-28 22:34:25 - WSIPredictor - INFO -    ✓ Extracted 27789 instances from tiles (before cleaning)
2026-01-28 22:34:25 - WSIPredictor - INFO -    🔍 DEBUG: Tile configuration:
2026-01-28 22:34:25 - WSIPredictor - INFO -       - Tile size: 512px
2026-01-28 22:34:25 - WSIPredictor - INFO -       - Overlap: 64px
2026-01-28 22:34:25 - WSIPredictor - INFO -       - Grid: 11x11 tiles
2026-01-28 22:34:25 - WSIPredictor - INFO -    🔍 DEBUG: Cells near tile boundaries (within 64px): 10189
2026-01-28 22:34:25 - WSIPredictor - INFO -    🔍 DEBUG: Potential overlap rate: 36.7%
2026-01-28 22:34:25 - WSIPredictor - INFO - 🧹 Cleaning overlapping 


RESULTS
✅ Found 16031 cells in 57.65s
   Output saved to: results_new


In [8]:
import torch
import numpy as np
import tifffile
from vitaminp import VitaminPFlex
from vitaminp.inference import WSIPredictor, ChannelConfig

# ============================================================================
# LOAD XENIUM DATA
# ============================================================================
print("="*60)
print("READING ORIGINAL XENIUM CHANNELS")
print("="*60)

data_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028"
morphology_focus_dir = f"{data_dir}/morphology_focus"

# Read DAPI channel
dapi_path = f"{morphology_focus_dir}/ch0000_dapi.ome.tif"
print(f"\nReading DAPI: {dapi_path}")
with tifffile.TiffFile(dapi_path) as tif:
    dapi = tif.pages[0].asarray()
    print(f"  ✅ DAPI loaded: {dapi.shape}, dtype={dapi.dtype}")

# Read CD45/E-cadherin channel
cd45_path = f"{morphology_focus_dir}/ch0001_atp1a1_cd45_e-cadherin.ome.tif"
print(f"Reading CD45/E-cadherin: {cd45_path}")
with tifffile.TiffFile(cd45_path) as tif:
    cd45_ecadherin = tif.pages[0].asarray()
    print(f"  ✅ CD45/E-cadherin loaded: {cd45_ecadherin.shape}, dtype={cd45_ecadherin.dtype}")

# Stack into 2-channel image (channels, height, width)
combined = np.stack([dapi, cd45_ecadherin], axis=0)
print(f"\n✅ Combined shape: {combined.shape} (channels, height, width)")

# Save as temporary OME-TIFF for WSIPredictor
import tempfile
temp_image_path = tempfile.mktemp(suffix='.ome.tiff')
tifffile.imwrite(temp_image_path, combined, photometric='minisblack')
print(f"✅ Saved temporary file: {temp_image_path}")

# ============================================================================
# SETUP MODEL & PREDICTOR
# ============================================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nUsing device: {device}")

# Load Model
print("\nLoading model...")
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold21_best.pth", map_location=device))
model.eval()
print("✅ Model loaded")

# Create channel config
config = ChannelConfig(
    nuclear_channel=0,           # Channel 0: DAPI
    membrane_channel=1,          # Channel 1: CD45/E-cadherin
    channel_names={0: 'DAPI', 1: 'CD45_Ecadherin'}
)

# Create predictor
predictor = WSIPredictor(
    model=model,
    device=device,
    patch_size=512,
    overlap=64,
    target_mpp=0.2125,          # Xenium pixel size
    magnification=40,
    mif_channel_config=config
)

print("\nPredictor settings:")
print(f"  Patch size: 512")
print(f"  Overlap: 64")
print(f"  Target MPP: 0.2125 μm/pixel")
print(f"  Magnification: 40x")

# ============================================================================
# RUN INFERENCE
# ============================================================================
print(f"\n{'='*60}")
print("RUNNING INFERENCE...")
print(f"{'='*60}")

results = predictor.predict(
    wsi_path=temp_image_path,
    output_dir='results_xenium_fullwsi',
    branch='mif_cell',              # Use cell branch for whole cells
    filter_tissue=True,
    tissue_threshold=0.01,
    clean_overlaps=True,            # Clean boundary artifacts
    save_geojson=True,              # Save GeoJSON for visualization
    detection_threshold=0.5,
    min_area_um=20.0,               # Filter small artifacts (10 μm²)
    mpp_override=0.263, 
    # ============ NEW PARAMETERS ============
    simplify_epsilon=None,         # ← Moderate simplification (try 0.5, 1.0, 2.0, or None)
    coord_precision=None,            # ← Round to 0.1 pixel (1 decimal place)
    save_parquet=True,            # ← Also save as .parquet format
    # ========================================
)

print(f"\n{'='*60}")
print("RESULTS")
print(f"{'='*60}")
print(f"✅ Found {results['num_detections']} cells in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")

# Clean up temp file
import os
os.remove(temp_image_path)
print(f"✅ Cleaned up temporary file")

READING ORIGINAL XENIUM CHANNELS

Reading DAPI: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028/morphology_focus/ch0000_dapi.ome.tif
  ✅ DAPI loaded: (112134, 54229), dtype=uint16
Reading CD45/E-cadherin: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028/morphology_focus/ch0001_atp1a1_cd45_e-cadherin.ome.tif
  ✅ CD45/E-cadherin loaded: (112134, 54229), dtype=uint16

✅ Combined shape: (2, 112134, 54229) (channels, height, width)
✅ Saved temporary file: /tmp/tmpukvj0ewl.ome.tiff

Using device: cuda

Loading model...
✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


2026-01-30 17:47:57 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-30 17:47:57 - WSIPredictor - INFO -   Device: cuda
2026-01-30 17:47:57 - WSIPredictor - INFO -   Model type: VitaminPFlex (single-modality)
2026-01-30 17:47:57 - WSIPredictor - INFO -   Patch size: 512
2026-01-30 17:47:57 - WSIPredictor - INFO -   Overlap: 64
2026-01-30 17:47:57 - WSIPredictor - INFO -   Magnification: 40
2026-01-30 17:47:57 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: CD45_Ecadherin
2026-01-30 17:47:57 - WSIPredictor - INFO -    Manual MPP override: 0.2630 μm/px
2026-01-30 17:47:57 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-30 17:47:57 - WSIPredictor - INFO -    WSI MPP: 0.2630 μm/px
2026-01-30 17:47:57 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-30 17:47:57 - WSIPredictor - INFO -    Scale factor: 1.00x
2026-01-30 17:47:57 - WSIPredictor - INFO -    Min area filter: 20.0 μm² = 289 pixels²
2026-01-30 17:47:57 - WSIPredictor - INFO - 📁 O

✅ Model loaded

Predictor settings:
  Patch size: 512
  Overlap: 64
  Target MPP: 0.2125 μm/pixel
  Magnification: 40x

RUNNING INFERENCE...


2026-01-30 17:48:42 - WSIPredictor - INFO -    ✓ MIF Size: 112134x54229 pixels, 2 channels
2026-01-30 17:48:42 - WSIPredictor - INFO - 📐 Extracting tiles...


Output: shape=(2, 112134, 54229), dtype=float32, range=[0.000, 0.070]
   Scanning 251x121 tile grid...


2026-01-30 17:50:20 - WSIPredictor - INFO -    ✓ Created 30371 tiles (251x121 grid)
2026-01-30 17:50:20 - WSIPredictor - INFO -    ✓ Tissue tiles: 23096/30371 (76.0%)
2026-01-30 17:50:20 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on mif_cell...


   Tissue dilation: 20137 → 23096 tiles (+2959 boundary tiles)


Processing tiles:  52%|█████▏    | 15646/30371 [24:53<23:25, 10.48it/s]   


KeyboardInterrupt: 

In [9]:
import tifffile
import numpy as np
import os

# Path to QPTIFF file
qptiff_path = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/57986/Scan1/57986_Scan1.er.qptiff"

print("Reading QPTIFF channels...")

with tifffile.TiffFile(qptiff_path) as tif:
    print(f"Total channels/pages: {len(tif.pages)}")
    
    # Read the three channels
    dapi = tif.pages[0].asarray()  # C1 (index 0)
    ecadherin = tif.pages[10].asarray()  # C11 (index 10)
    cd45 = tif.pages[16].asarray()  # C17 (index 16)
    
    print(f"\nFull image shape: {dapi.shape}")
    print(f"DAPI dtype: {dapi.dtype}, range: [{dapi.min()}, {dapi.max()}]")
    print(f"E-cadherin dtype: {ecadherin.dtype}, range: [{ecadherin.min()}, {ecadherin.max()}]")
    print(f"CD45 dtype: {cd45.dtype}, range: [{cd45.min()}, {cd45.max()}]")

# Extract center 3000x3000 tile
h, w = dapi.shape
center_h, center_w = h // 2, w // 2
tile_size = 3000

print(f"\nExtracting {tile_size}x{tile_size} center tile...")

dapi_tile = dapi[center_h-tile_size//2:center_h+tile_size//2, 
                 center_w-tile_size//2:center_w+tile_size//2]
ecad_tile = ecadherin[center_h-tile_size//2:center_h+tile_size//2, 
                      center_w-tile_size//2:center_w+tile_size//2]
cd45_tile = cd45[center_h-tile_size//2:center_h+tile_size//2, 
                 center_w-tile_size//2:center_w+tile_size//2]

print(f"Tile shape: {dapi_tile.shape}")

# Create output directory
output_dir = "test_images"
os.makedirs(output_dir, exist_ok=True)

# Option 1: DAPI + E-cadherin
print("\nSaving Option 1: DAPI + E-cadherin...")
combined_ecad = np.stack([dapi_tile, ecad_tile], axis=0)
output_path_1 = os.path.join(output_dir, "tile_dapi_ecadherin.ome.tif")
tifffile.imwrite(output_path_1, combined_ecad, photometric='minisblack')
print(f"  ✅ Saved: {output_path_1}")
print(f"     Shape: {combined_ecad.shape}, dtype: {combined_ecad.dtype}")

# Option 2: DAPI + CD45
print("\nSaving Option 2: DAPI + CD45...")
combined_cd45 = np.stack([dapi_tile, cd45_tile], axis=0)
output_path_2 = os.path.join(output_dir, "tile_dapi_cd45.ome.tif")
tifffile.imwrite(output_path_2, combined_cd45, photometric='minisblack')
print(f"  ✅ Saved: {output_path_2}")
print(f"     Shape: {combined_cd45.shape}, dtype: {combined_cd45.dtype}")

# Option 3: DAPI + Merged membrane (E-cadherin + CD45)
print("\nSaving Option 3: DAPI + Merged membrane...")
membrane_merged = np.maximum(ecad_tile, cd45_tile)
combined_merged = np.stack([dapi_tile, membrane_merged], axis=0)
output_path_3 = os.path.join(output_dir, "tile_dapi_merged.ome.tif")
tifffile.imwrite(output_path_3, combined_merged, photometric='minisblack')
print(f"  ✅ Saved: {output_path_3}")
print(f"     Shape: {combined_merged.shape}, dtype: {combined_merged.dtype}")

# Also save individual channels for reference
print("\nSaving individual channels...")
tifffile.imwrite(os.path.join(output_dir, "tile_dapi_only.tif"), dapi_tile)
tifffile.imwrite(os.path.join(output_dir, "tile_ecadherin_only.tif"), ecad_tile)
tifffile.imwrite(os.path.join(output_dir, "tile_cd45_only.tif"), cd45_tile)
print(f"  ✅ Saved individual channel TIFs")

print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"✅ All tiles saved to: {output_dir}/")
print(f"   - tile_dapi_ecadherin.ome.tif (Option 1)")
print(f"   - tile_dapi_cd45.ome.tif (Option 2)")
print(f"   - tile_dapi_merged.ome.tif (Option 3)")
print(f"   - Individual channel TIFs for reference")
print(f"\nTile size: 3000×3000 pixels")
print(f"Data type: uint16 (preserved from original)")
print(f"Center location: ({center_h}, {center_w})")

Reading QPTIFF channels...
Total channels/pages: 369

Full image shape: (51120, 27840)
DAPI dtype: uint16, range: [0, 65395]
E-cadherin dtype: uint16, range: [0, 64039]
CD45 dtype: uint16, range: [0, 64608]

Extracting 3000x3000 center tile...
Tile shape: (3000, 3000)

Saving Option 1: DAPI + E-cadherin...
  ✅ Saved: test_images/tile_dapi_ecadherin.ome.tif
     Shape: (2, 3000, 3000), dtype: uint16

Saving Option 2: DAPI + CD45...
  ✅ Saved: test_images/tile_dapi_cd45.ome.tif
     Shape: (2, 3000, 3000), dtype: uint16

Saving Option 3: DAPI + Merged membrane...
  ✅ Saved: test_images/tile_dapi_merged.ome.tif
     Shape: (2, 3000, 3000), dtype: uint16

Saving individual channels...
  ✅ Saved individual channel TIFs

SUMMARY
✅ All tiles saved to: test_images/
   - tile_dapi_ecadherin.ome.tif (Option 1)
   - tile_dapi_cd45.ome.tif (Option 2)
   - tile_dapi_merged.ome.tif (Option 3)
   - Individual channel TIFs for reference

Tile size: 3000×3000 pixels
Data type: uint16 (preserved from or

## dual Xenium

In [ ]:
import torch
import numpy as np
import tifffile
from vitaminp import VitaminPDual
from vitaminp.inference import WSIPredictor, ChannelConfig

# ============================================================================
# LOAD XENIUM DATA
# ============================================================================
print("="*60)
print("READING XENIUM CHANNELS")
print("="*60)

data_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028"
morphology_focus_dir = f"{data_dir}/morphology_focus"

# Read DAPI channel
dapi_path = f"{morphology_focus_dir}/ch0000_dapi.ome.tif"
print(f"\nReading DAPI: {dapi_path}")
with tifffile.TiffFile(dapi_path) as tif:
    dapi = tif.pages[0].asarray()
    print(f"  ✅ DAPI loaded: {dapi.shape}, dtype={dapi.dtype}")

# Read CD45/E-cadherin channel
cd45_path = f"{morphology_focus_dir}/ch0001_atp1a1_cd45_e-cadherin.ome.tif"
print(f"Reading CD45/E-cadherin: {cd45_path}")
with tifffile.TiffFile(cd45_path) as tif:
    cd45_ecadherin = tif.pages[0].asarray()
    print(f"  ✅ CD45/E-cadherin loaded: {cd45_ecadherin.shape}, dtype={cd45_ecadherin.dtype}")

# Stack into 2-channel MIF image (channels, height, width)
mif_combined = np.stack([cd45_ecadherin, dapi], axis=0)
print(f"\n✅ MIF Combined shape: {mif_combined.shape} (channels, height, width)")

# Save as temporary OME-TIFF for WSIPredictor
import tempfile
temp_mif_path = tempfile.mktemp(suffix='_mif.ome.tiff')
tifffile.imwrite(temp_mif_path, mif_combined, photometric='minisblack')
print(f"✅ Saved temporary MIF file: {temp_mif_path}")

# H&E path
he_path = f"{data_dir}/registration/Lung_Xenium_1_40X_rescan_registered.ome.tif"
print(f"\n✅ H&E path: {he_path}")

# ============================================================================
# SETUP DUAL MODEL & PREDICTOR
# ============================================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n{'='*60}")
print(f"DUAL MODEL SETUP")
print(f"{'='*60}")
print(f"Using device: {device}")

# Load Dual Model
print("\nLoading VitaminP Dual model...")
model = VitaminPDual(model_size='large').to(device)
model.load_state_dict(
    torch.load("checkpoints/vitamin_p_dual_large_fold21_best.pth", map_location=device)
)
model.eval()
print("✅ Dual model loaded")

# Create channel config for MIF
config = ChannelConfig(
    nuclear_channel=1,           # DAPI is in channel 1
    membrane_channel=[0],        # CD45/E-cadherin in channel 0
    membrane_combination='max',  # Use max if multiple channels
    channel_names={0: 'CD45_Ecadherin', 1: 'DAPI'}
)

# Create predictor
predictor = WSIPredictor(
    model=model,
    device=device,
    patch_size=512,
    overlap=64,
    target_mpp=0.2125,          # Xenium pixel size
    magnification=40,
    mif_channel_config=config
)

print("\nPredictor settings:")
print(f"  Patch size: 512")
print(f"  Overlap: 64")
print(f"  Target MPP: 0.2125 μm/pixel (Xenium)")
print(f"  Magnification: 40x")
print(f"  Nuclear channel: 1 (DAPI)")
print(f"  Membrane channel: 0 (CD45/E-cadherin)")

# ============================================================================
# RUN DUAL INFERENCE (H&E + MIF)
# ============================================================================
print(f"\n{'='*60}")
print("RUNNING DUAL INFERENCE...")
print(f"{'='*60}")

results = predictor.predict(
    wsi_path=he_path,                      # H&E image
    wsi_path_mif=temp_mif_path,           # MIF image
    output_dir='results_xenium_dual',
    branches=['he_nuclei', 'he_cell', 'mif_nuclei', 'mif_cell'],
    filter_tissue=True,
    tissue_threshold=0.01,
    clean_overlaps=True,
    save_geojson=True,
    save_visualization=True,
    detection_threshold=0.5,
    min_area_um=20.0,
    mpp_override=0.263,
    simplify_epsilon=None,
    coord_precision=None,
    save_parquet=True,
)

# ============================================================================
# DISPLAY RESULTS
# ============================================================================
print(f"\n{'='*60}")
print("RESULTS")
print(f"{'='*60}")

print(f"\n📊 H&E Results:")
print(f"  ✅ Nuclei detected: {results['he_nuclei']['num_detections']}")
print(f"  ✅ Cells detected: {results['he_cell']['num_detections']}")

print(f"\n📊 MIF Results:")
print(f"  ✅ Nuclei detected: {results['mif_nuclei']['num_detections']}")
print(f"  ✅ Cells detected: {results['mif_cell']['num_detections']}")

print(f"\n⏱️  Processing time: {results.get('processing_time', 'N/A')}")
print(f"📁 Output directory: {results.get('output_dir', 'N/A')}")

# Clean up temp file
import os
os.remove(temp_mif_path)
print(f"\n✅ Cleaned up temporary file")

READING XENIUM CHANNELS

Reading DAPI: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028/morphology_focus/ch0000_dapi.ome.tif
  ✅ DAPI loaded: (112134, 54229), dtype=uint16
Reading CD45/E-cadherin: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028/morphology_focus/ch0001_atp1a1_cd45_e-cadherin.ome.tif
  ✅ CD45/E-cadherin loaded: (112134, 54229), dtype=uint16

✅ MIF Combined shape: (2, 112134, 54229) (channels, height, width)
✅ Saved temporary MIF file: /tmp/tmpq5qo4x7y_mif.ome.tiff

✅ H&E path: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028/registration/Lung_Xenium_1_40X_rescan_registered.ome.tif

DUAL MODEL SETUP
Using device: cuda

Loading VitaminP Dual model...
Building H&E encoder with DINOv2-large
Building MIF encoder with DINOv2-large
Building shared encod

2026-01-30 18:19:57 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-30 18:19:57 - WSIPredictor - INFO -   Device: cuda
2026-01-30 18:19:57 - WSIPredictor - INFO -   Model type: VitaminPDual (dual-modality)
2026-01-30 18:19:57 - WSIPredictor - INFO -   Patch size: 512
2026-01-30 18:19:57 - WSIPredictor - INFO -   Overlap: 64
2026-01-30 18:19:57 - WSIPredictor - INFO -   Magnification: 40
2026-01-30 18:19:57 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: max(CD45_Ecadherin)
2026-01-30 18:19:57 - WSIPredictor - INFO - 
2026-01-30 18:19:57 - WSIPredictor - INFO - Processing branch: he_nuclei
2026-01-30 18:19:57 - WSIPredictor - INFO - ============================================================
2026-01-30 18:19:57 - WSIPredictor - INFO - 🔄 Using MIF predictions for he_nuclei (better quality)
2026-01-30 18:19:57 - WSIPredictor - INFO -    Manual MPP override: 0.2630 μm/px
2026-01-30 18:19:57 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-30 18:19:57 - WSI

✅ Dual model loaded

Predictor settings:
  Patch size: 512
  Overlap: 64
  Target MPP: 0.2125 μm/pixel (Xenium)
  Magnification: 40x
  Nuclear channel: 1 (DAPI)
  Membrane channel: 0 (CD45/E-cadherin)

RUNNING DUAL INFERENCE...


2026-01-30 18:20:56 - WSIPredictor - INFO -    ✓ MIF Size: 112134x54229 pixels, 2 channels
2026-01-30 18:20:56 - WSIPredictor - INFO - 📐 Extracting tile positions...


Output: shape=(2, 112134, 54229), dtype=float32, range=[0.000, 0.070]
   Virtual upscaled size: 112134x54229 (from 112134x54229)
   Scanning 251x121 tile grid...


...

2026-01-30 23:00:40 - WSIPredictor - INFO - ✅ Complete! 983732 detections in 8625.07s
2026-01-30 23:00:47 - WSIPredictor - INFO - 
2026-01-30 23:00:47 - WSIPredictor - INFO - Processing branch: mif_nuclei
2026-01-30 23:00:47 - WSIPredictor - INFO - ============================================================
2026-01-30 23:00:47 - WSIPredictor - INFO -    Manual MPP override: 0.2630 μm/px
2026-01-30 23:00:47 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-30 23:00:47 - WSIPredictor - INFO -    WSI MPP: 0.2630 μm/px
2026-01-30 23:00:47 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-30 23:00:47 - WSIPredictor - INFO -    Scale factor: 1.00x
2026-01-30 23:00:47 - WSIPredictor - INFO -    Min area filter: 20.0 μm² = 289 pixels²
2026-01-30 23:00:47 - WSIPredictor - INFO - 📁 Opening dual WSI pair:
2026-01-30 23:00:47 - WSIPredictor - INFO -    H&E: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__

Output: shape=(2, 112134, 54229), dtype=float32, range=[0.000, 0.070]
   Virtual upscaled size: 112134x54229 (from 112134x54229)
   Scanning 251x121 tile grid...


2026-01-30 23:09:21 - WSIPredictor - INFO -    ✓ Created 30371 tiles (251x121 grid)
2026-01-30 23:09:21 - WSIPredictor - INFO -    ✓ Tissue tiles: 30248/30371 (99.6%)
2026-01-30 23:09:21 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on mif_nuclei...


   Tissue dilation: 27659 → 30248 tiles (+2589 boundary tiles)


Processing tiles:   0%|          | 104/30371 [00:13<1:11:31,  7.05it/s]